In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from cellpose_omni import io
import skimage.io
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300
from cellpose_omni import dynamics, transforms
from scipy import stats


In [ ]:
from aicsimageio.readers import CziReader
import os
basedir = '/home/kcutler/DataDrive/confocal'
dataset = 'Experiment-712' #720 too dim, 733 too thick, 704 too binned,707 good! try 708 next 
dataset = 'Experiment-707' 
dataset = 'Experiment-719' #clean up edges of my dataset

reader = CziReader(os.path.join(basedir,dataset+'.czi'))
im = reader.data.squeeze()

In [ ]:
# import tifffile
# im = tifffile.imread('/home/kcutler/DataDrive/3D_BBBC/BBBC027/train/0000.tif')

In [ ]:
modeldir = '/home/kcutler/DataDrive/confocal/train60/models/cellpose_residual_on_style_on_concatenation_off_omni_train60_2022_04_28_02_39_03.970940_epoch_3999'
modeldir = '/home/kcutler/DataDrive/confocal/train60/models/cellpose_residual_on_style_on_concatenation_off_omni_train60_2022_04_28_02_39_03.970940_epoch_3951'

In [ ]:
dim = 3
nclasses = dim+2
nchan = 1
omni = 1
rescale = False
diam_mean = 0
from cellpose_omni import models, core
use_GPU = core.use_gpu()
print('>>> GPU activated? %d'%use_GPU)
use_GPU = 0
model = models.CellposeModel(gpu=use_GPU, pretrained_model=modeldir, net_avg=False, 
                             diam_mean=diam_mean, nclasses=nclasses, dim=dim, nchan=nchan)
model.pretrained_model

In [ ]:
mask_threshold = -5 ##############
diam_threshold = 12
net_avg = 0
cluster = 0
verbose = 1

tile = 0
chans = [0,0]
chans = None
compute_masks = 1
slc_crop = slice(None)
# slc_crop = slice(100, 120, None)
# slc_crop = slice(13, 53, None) # 733
# slc_crop = slice(50, 100, None) # 704
slc_crop = slice(15, 52, None) # 707
# slc_crop = slice(30, 70, None) # 712
# slc_crop = (Ellipsis,)+tuple([slice(0,129)]*2)
resample=False
rescale = None

imcrop = im[slc_crop]
# imcrop = im[100:120]
# imcrop = im.copy()
# import torch
imcrop.shape

In [ ]:
masks, flows, styles = model.eval(imcrop,channels = chans,rescale=rescale,mask_threshold=mask_threshold,net_avg=net_avg,
                                  transparency=True,flow_threshold=0.,omni=omni,resample=resample,verbose=verbose,
                                  diam_threshold=diam_threshold,cluster=cluster,tile=0,
                                  compute_masks=compute_masks)
#     else:

In [ ]:
import omnipose, tifffile
mu = flows[1]
dt = flows[2]
p = flows[3]
bd = flows[4]
suffix = '_1951'
ext = '.tif'
tifffile.imwrite(os.path.join(basedir,dataset+'_trained_flow'+suffix+ext),mu)
tifffile.imwrite(os.path.join(basedir,dataset+'_trained_dist'+suffix+ext),dt)


In [ ]:
hdbscan = 1
cluster = 0
masks,p,_= omnipose.core.compute_masks(mu,dt,bd=bd,mask_threshold=mask_threshold,
                                           use_gpu=True,verbose=True,cluster=cluster, 
                                           flow_threshold=0., dim=3, hdbscan=hdbscan, 
                                        debug=0, eps=3, min_size=30, nclasses=im.ndim+2)

In [ ]:
import napari, ncolor
from omnipose.utils import sinebow

In [ ]:
mnc = ncolor.label(masks)
N = np.max(mnc)
viewer = napari.view_image(imcrop, name='cells')
viewer.add_labels(mnc, name='labels',color=sinebow(N+3))

In [ ]:
import tifffile
tifffile.imwrite(os.path.join(basedir,dataset+'_trained_masks'),masks)
tifffile.imwrite(os.path.join(basedir,dataset+'_trained_flow'),mu)
tifffile.imwrite(os.path.join(basedir,dataset+'_trained_dist'),dt)



In [ ]:
viewer.add_image(flows[2], name='dist')
viewer.add_image(flows[4], name='bd')
viewer.add_image(flows[0], name='flow')


In [ ]:
mu = flows[1]
mag = np.sqrt(np.nansum(mu**2,axis=0))
viewer.add_image(mag>2.5)

In [ ]:
gt = ncolor.label(tifffile.imread('/home/kcutler/DataDrive/3D_BBBC/BBBC027/train/0000_masks.tif'))[slc_crop]
N = gt.max()
viewer.add_labels(gt, name='labels_gt',color=sinebow(N+3))

In [ ]:
mag.max()

In [ ]:
im.shape

In [ ]:
plt.imshow(im[:,300])

In [ ]:
import tifffile
stack = tifffile.imread('/home/kcutler/DataDrive/luca/data/tif_stacks/191.tif')

In [ ]:
viewer = napari.view_image(stack, name='cells')


In [ ]:
from aicsimageio import AICSImage
idir = '/home/kcutler/DataDrive/3D_BBBC/BBBC024_v1_c75_highSNR_foreground/image-labels_0015.ics'
img = AICSImage(idir)


In [ ]:
# img.data  # returns 5D TCZYX numpy array
# img.xarray_data  # returns 5D TCZYX xarray data array backed by numpy
# img.dims  # returns a Dimensions object
# img.dims.order  # returns string "TCZYX"
# img.dims.X  # returns size of X dimension
# img.shape  # returns tuple of dimension sizes in TCZYX order
# img.get_image_data("CZYX", T=0)  # returns 4D CZYX numpy array
import napari
viewer = napari.view_image(img.data.squeeze(), name='cells')


In [ ]:
!pip install aicsimageio[bfio] --force --upgrade

In [ ]:
import numpy as np
import tifffile
from cloudvolume import CloudVolume

# vol = CloudVolume(
#     "s3://open-neurodata/hildebrand/120809zf104/151009", mip=0, use_https=True
# )

# # load data into numpy array
# cutout = vol[1024:1536, 512:1024, 7520:7536]


vol = CloudVolume(
    "s3://open-neurodata/cardona/acardona_abd1_5/image", mip=0, use_https=True
)

# load data into numpy array
cutout = vol[10752:11264, 9216:9728, 96:112]
viewer = napari.view_image(cutout.squeeze(), name='cells')


In [ ]:
mask = tifffile.imread('/home/kcutler/DataDrive/Pedicel/Pedicel Segmentation.tif')
im = tifffile.imread('/home/kcutler/DataDrive/Pedicel/Pedicel stack.tif')

In [ ]:
import napari
# mnc = ncolor.label(mask)
# N = np.max(mnc)
viewer = napari.view_image(im, name='cells')
viewer.add_labels(np.flip(mask,axis=0), name='labels')

In [ ]:
cutout.shape

In [ ]:
import ncolor
mnc0 = ncolor.label(mask,max_depth = 11)

In [ ]:
mask.shape,im.shape

In [ ]:
viewer.add_labels(np.flip(mask,axis=2), name='labels2')

In [ ]:
viewer.add_labels(np.flip(mask,axis=1), name='labels1')

In [ ]:
from cellpose_omni import plot
fl = plot.dx_to_circ(flows[1])

In [ ]:
from cellpose_omni import io
save0 = os.path.join('/home/kcutler/DataDrive/confocal',dataset,'flow')
io.check_dir(save0)
ext = '.png'
for k in range(im.shape[1]):
# for k in [0]:
    # imk = np.uint8(flows[0][:,k])
    imk = np.uint8(fl[:,k])
    skimage.io.imsave(os.path.join(save0,str(k)+ext),imk)

In [ ]:
flows[1].shape

In [ ]:
plt.imshow(fl[:,4])

In [ ]:
2755/10*40/60/60